[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fga-0/automating_data_augmentation/blob/main/wrn_concat_raw_aug.ipynb)

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim

import utils
import importlib
importlib.reload(utils)

<module 'utils' from '/content/utils.py'>

In [2]:
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path_to_data = "drive/MyDrive/SDD/data_augmentation/cifar10"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
transform_augment = transforms.Compose([
    transforms.Resize(224),
    AutoAugment(AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
    # transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root=path_to_data, train=True, download=True, transform=transform)
trainset_aug = torchvision.datasets.CIFAR10(root=path_to_data, train=True, download=True, transform=transform_augment)
testset = torchvision.datasets.CIFAR10(root=path_to_data, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_subset_aug, test_subset_aug = utils.get_subset(trainset=trainset_aug, testset=testset, percentage=0.01)

train_subset, test_subset = utils.get_subset(trainset=trainset, testset=testset, percentage=0.02)

Training samples: 500
Test samples: 100
Training samples: 1000
Test samples: 200


In [6]:
from torch.utils.data import ConcatDataset

In [7]:
train_subset_raw_and_AA_CIFAR = ConcatDataset([train_subset, train_subset_aug])
train_loader_AA_CIFAR = DataLoader(train_subset_raw_and_AA_CIFAR, batch_size=32, shuffle=True, num_workers=2)

# test_loader_AA_CIFAR = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=2) # inutile d'utiliser ce test loader ?
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

In [9]:
from torchvision.models import wide_resnet50_2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# Load Wide ResNet model
# wrn = wide_resnet50_2(num_classes=10)  # CIFAR-10 has 10 classes

# Load a pre-trained Wide ResNet-50-2 model:
wrn = wide_resnet50_2(weights='DEFAULT')
# Freeze all model parameters except for the final layer:
for param in wrn.parameters():
    param.requires_grad = False
# Get the number of input features for the original last layer:
num_feature = wrn.fc.in_features
# Replace the final classification layer to match your dataset:
wrn.fc = nn.Linear(num_feature, 10)
# View the structure of the new final layer (optional):
print(wrn.fc)
# Move the model to the GPU for accelerated training:
wrn = wrn.to(device)

print(f"Using device: {device}")

Linear(in_features=2048, out_features=10, bias=True)
Using device: cuda


In [17]:
criterion = nn.CrossEntropyLoss()

In [18]:
optimizer = optim.Adam(wrn.fc.parameters(), lr=0.001)
# Learning rate scheduler for better convergence
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [19]:
wrn_trained_on_raw_AND_augmented_data = utils.train_WideResNet(model=wrn, trainloader=train_loader_AA_CIFAR, num_epochs=5, batch_size=32, optimizer=optimizer, criterion=criterion, device=device, scheduler=scheduler)

Epoch 1: Loss=1.8184, Accuracy=45.20%


Epoch 2: Loss=1.2103, Accuracy=69.93%


Epoch 3: Loss=0.9732, Accuracy=74.60%


Epoch 4: Loss=0.8960, Accuracy=74.53%


Epoch 5: Loss=0.7931, Accuracy=76.80%
Training complete!


In [20]:
utils.evaluate(wrn_trained_on_raw_AND_augmented_data, test_loader, device)

Accuracy: 78.50%
Taux d'erreur: 21.50%


(0.785, 0.21499999999999997)

# WRN RAW

In [12]:
# Load a pre-trained Wide ResNet-50-2 model:
wrn_raw = wide_resnet50_2(weights='DEFAULT')
# Freeze all model parameters except for the final layer:
for param in wrn_raw.parameters():
    param.requires_grad = False
# Get the number of input features for the original last layer:
num_feature = wrn_raw.fc.in_features
# Replace the final classification layer to match your dataset:
wrn_raw.fc = nn.Linear(num_feature, 10)
# View the structure of the new final layer (optional):
print(wrn_raw.fc)
# Move the model to the GPU for accelerated training:
wrn_raw = wrn_raw.to(device)

Linear(in_features=2048, out_features=10, bias=True)


In [13]:
optimizer = optim.Adam(wrn_raw.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [14]:
train_loader = DataLoader(train_subset, batch_size=32, shuffle=False)
wrn_trained_on_raw_data = utils.train_WideResNet(model=wrn_raw, trainloader=train_loader, num_epochs=5, batch_size=32, optimizer=optimizer, criterion=criterion, device=device, scheduler=scheduler)

Epoch 1: Loss=1.9902, Accuracy=34.90%


Epoch 2: Loss=1.3392, Accuracy=69.80%


Epoch 3: Loss=1.0346, Accuracy=77.60%


Epoch 4: Loss=0.8499, Accuracy=82.30%


Epoch 5: Loss=0.7218, Accuracy=85.40%
Training complete!


In [15]:
utils.evaluate(wrn_trained_on_raw_data, test_loader, device)

Accuracy: 62.50%
Taux d'erreur: 37.50%


(0.625, 0.375)